In [117]:
# !pip install haversine

In [118]:
import pandas as pd
import haversine

In [2]:
df = pd.read_json('artists_shows_shard.json', lines=True)

In [5]:
print(df.shape)
df.head(3)

(15729, 3)


,artist,date,location
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"


The column named loc should be renamed to prevent confusion  
with the 'pandas.DataFrame.loc()' method

In [9]:
df.columns = ['artist', 'date', 'location']
df.head(3)

,artist,date,location
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"


Trim shows outside US

In [10]:
domestic_df = df[df['location'].str.endswith('US')]
print(domestic_df.shape)
domestic_df.head(3)

(9608, 3)


,artist,date,location
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"
3,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US"


Lets only make decisions based on shows with in the past five years.  
Trim any concerts before 2015.

In [30]:
recent_domestic_df = domestic_df[domestic_df.date >= '2015'].copy()
print(recent_domestic_df.shape)

(8642, 3)


### Lets merge/join some lat long info to these locations so we can compare them!

With some help from our friends at https://simplemaps.com/data/us-cities:  
We can get a csv of cities and their latititude and longitude from some census.

In [12]:
cities_df = pd.read_csv('uscitiesv1.5.csv')

In [13]:
print(cities_df.shape)
cities_df.head(2)

(37842, 16)


,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314


Let's just grab city, state id, lat and lng.

In [14]:
cities_df = cities_df[['city', 'state_id', 'lat', 'lng']]
cities_df.head(2)

,city,state_id,lat,lng
0,Prairie Ridge,WA,47.1443,-122.1408
1,Edison,WA,48.5602,-122.4311


Lets adjust these city and state values so they will match

In [15]:
cities_df.loc[:,'comb_city_state'] = cities_df['city'] + ', ' + cities_df['state_id'] + ', US'

In [16]:
cities_df.head()

,city,state_id,lat,lng,comb_city_state
0,Prairie Ridge,WA,47.1443,-122.1408,"Prairie Ridge, WA, US"
1,Edison,WA,48.5602,-122.4311,"Edison, WA, US"
2,Packwood,WA,46.6085,-121.6702,"Packwood, WA, US"
3,Wautauga Beach,WA,47.5862,-122.5482,"Wautauga Beach, WA, US"
4,Harper,WA,47.5207,-122.5196,"Harper, WA, US"


In [17]:
combined = pd.merge(left=recent_domestic_df, right=cities_df,
             left_on='location', right_on='comb_city_state', how='left')

In [18]:
combined.head()

,artist,date,location,city,state_id,lat,lng,comb_city_state
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US",Las Vegas,NV,36.2291,-115.2607,"Las Vegas, NV, US"
1,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US",Anaheim,CA,33.8390,-117.8572,"Anaheim, CA, US"
2,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US",Atlanta,GA,33.7626,-84.4228,"Atlanta, GA, US"
3,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US",Las Vegas,NV,36.2291,-115.2607,"Las Vegas, NV, US"
4,2 Chainz,2019-05-19 00:00:00,"Detroit, MI, US",Detroit,MI,42.3834,-83.1024,"Detroit, MI, US"


Confirm shape matches old shape, no DUPs

In [115]:
recent_domestic_df.shape[0] == combined.shape[0]

145

Are there any missing values?

In [19]:
print(combined.shape)
combined.isna().sum()

(8642, 8)


artist               0
date                 0
location             0
city               276
state_id           276
lat                276
lng                276
comb_city_state    276
dtype: int64

Where is it failing to connect?

In [20]:
combined[combined['city'].isna()][:18]

,artist,date,location,city,state_id,lat,lng,comb_city_state
56,2 Chainz,2017-09-16 00:00:00,"St Petersburg, FL, US",NaN,NaN,NaN,NaN,NaN
68,2 Chainz,2017-08-30 01:00:00,"St Louis, MO, US",NaN,NaN,NaN,NaN,NaN
71,2 Chainz,2017-08-26 02:30:00,"St. Paul, MN, US",NaN,NaN,NaN,NaN,NaN
86,2 Chainz,2017-04-23 23:00:00,"Amherst, MA, US",NaN,NaN,NaN,NaN,NaN
87,2 Chainz,2017-04-22 19:00:00,"Pozo, CA, US",NaN,NaN,NaN,NaN,NaN
108,21 Savage,2019-07-31 01:00:00,"St. Paul, MN, US",NaN,NaN,NaN,NaN,NaN
147,21 Savage,2018-05-27 00:00:00,"Darien Center, NY, US",NaN,NaN,NaN,NaN,NaN
148,21 Savage,2018-05-24 23:00:00,"Mansfield, MA, US",NaN,NaN,NaN,NaN,NaN
176,21 Savage,2017-12-09 02:00:00,"St Louis, MO, US",NaN,NaN,NaN,NaN,NaN
198,21 Savage,2017-09-15 15:45:00,"Flushing, NY, US",NaN,NaN,NaN,NaN,NaN


Won't get to all of these but probably can fix the 'saint's.  
How are they spelled in the csv?

In [21]:
cities_df[cities_df['city'].str.contains('Saint ')][:3]

,city,state_id,lat,lng,comb_city_state
657,Saint John,WA,47.0915,-117.5887,"Saint John, WA, US"
1232,Saint Charles,VA,36.8052,-83.0575,"Saint Charles, VA, US"
1592,Saint Paul,VA,36.9069,-82.3164,"Saint Paul, VA, US"


In [22]:
print(cities_df[cities_df['comb_city_state'].str.contains('Saint ')].shape)
print(cities_df[cities_df['comb_city_state'].str.contains('St. ')].shape)
print(cities_df[cities_df['comb_city_state'].str.contains('St ')].shape)

(249, 5)
(1, 5)
(0, 5)


In [23]:
cities_df[cities_df['city'].str.contains('St. ')]

,city,state_id,lat,lng,comb_city_state
33971,St. John,KS,38.0,-98.7611,"St. John, KS, US"


In the census csv, almost all the cities with 'Saint' in the name have it spelled out, not abbreviated ('St' or 'St.')  
Lets adjust the St John from the csv and then regularize these in our concert data

In [24]:
# Regularize cities csv "St. John Kansas"
cities_df.loc[:,'comb_city_state'] = cities_df['comb_city_state'].str.replace('St. ', 'Saint ')

In [25]:
cities_df[cities_df['comb_city_state'].str.contains('St. ')]

,city,state_id,lat,lng,comb_city_state


In [26]:
print(recent_domestic_df[recent_domestic_df['location'].str.contains('Saint ')].shape)
print(recent_domestic_df[recent_domestic_df['location'].str.contains('St. ')].shape)
print(recent_domestic_df[recent_domestic_df['location'].str.contains('St ')].shape)

(20, 3)
(58, 3)
(73, 3)


In [31]:
recent_domestic_df.loc[:, 'location'] = recent_domestic_df['location'].str.replace('St. ', 'St ')
recent_domestic_df.loc[:, 'location'] = recent_domestic_df['location'].str.replace('St ', 'Saint ')

In [33]:
display(recent_domestic_df[recent_domestic_df['location'].str.contains('St. ')])
display(recent_domestic_df[recent_domestic_df['location'].str.contains('St ')])

,artist,date,location


,artist,date,location


In [57]:
combined = pd.merge(left=recent_domestic_df, right=cities_df,
             left_on='location', right_on='comb_city_state', how='left')

In [58]:
print(combined.shape)
combined.isna().sum()

(8642, 8)


artist               0
date                 0
location             0
city               145
state_id           145
lat                145
lng                145
comb_city_state    145
dtype: int64

In [59]:
n = 145
N = 8642 + 145
print(f'{n} out of {N}, or {1- round(n/N,2)}%... Not bad')

145 out of 8787, or 0.98%... Not bad


In [71]:
combined.dropna(inplace=True)   # Getting rid of those nan's

In [72]:
combined.head()

,artist,date,location,city,state_id,lat,lng,comb_city_state,comb_lat_long
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US",Las Vegas,NV,36.2291,-115.2607,"Las Vegas, NV, US","36.2291, -115.2607"
1,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US",Anaheim,CA,33.8390,-117.8572,"Anaheim, CA, US","33.839, -117.8572"
2,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US",Atlanta,GA,33.7626,-84.4228,"Atlanta, GA, US","33.7626, -84.4228"
3,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US",Las Vegas,NV,36.2291,-115.2607,"Las Vegas, NV, US","36.2291, -115.2607"
4,2 Chainz,2019-05-19 00:00:00,"Detroit, MI, US",Detroit,MI,42.3834,-83.1024,"Detroit, MI, US","42.3834, -83.1024"


In [128]:
combined['comb_lat_long'] = combined.lat.astype(str) + ', ' + combined.lng.astype(str)
# combined['comb_lat_long'] = combined['comb_lat_long'].str.split(', ')

In [129]:
combined.head()

,artist,date,location,city,state_id,lat,lng,comb_city_state,comb_lat_long
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US",Las Vegas,NV,36.2291,-115.2607,"Las Vegas, NV, US","36.2291, -115.2607"
1,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US",Anaheim,CA,33.8390,-117.8572,"Anaheim, CA, US","33.839, -117.8572"
2,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US",Atlanta,GA,33.7626,-84.4228,"Atlanta, GA, US","33.7626, -84.4228"
3,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US",Las Vegas,NV,36.2291,-115.2607,"Las Vegas, NV, US","36.2291, -115.2607"
4,2 Chainz,2019-05-19 00:00:00,"Detroit, MI, US",Detroit,MI,42.3834,-83.1024,"Detroit, MI, US","42.3834, -83.1024"


### Can we iterate over an artist?

In [130]:
artist_latlongs[0] in artist_latlongs, type(artist_latlongs)

(True, numpy.ndarray)

In [131]:
artist_latlongs = combined.loc[combined['artist']=='2 Chainz', 'comb_lat_long'].unique()
dif_locs = []
for latlong in combined.loc[combined['artist']=='21 Savage','comb_lat_long'].unique():
    if latlong not in artist_latlongs:
        
    print(type(latlong), type(artist_latlongs), type(artist_latlongs[0]), latlong in artist_latlongs)
    

<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> False
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> False
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> False
<class 'str'> <class 'numpy.ndarray'> <class 'str'> False
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'numpy.ndarray'> <class 'str'> False
<class 'str'> <class 'numpy.ndarray'> <class 'str'> True
<class 'str'> <class 'nump

### Maybe aggregate into lists?

In [51]:
# recent_domestic_df['location'].map(lambda x: x+'::')
combined.groupby(by='artist', ).agg(sum)

,lat,lng
artist,,
2 Chainz,3421.0323,-9380.7433
21 Savage,4244.5076,-11053.4291
3 Doors Down,3369.3411,-8415.8413
3LAU,3593.5149,-9493.2779
5 Seconds of Summer,3198.3369,-7984.9836
50 Cent,3388.3718,-8604.0896
6LACK,2800.2853,-6939.3369
6ix9ine,66.4988,-214.4708
A Boogie Wit da Hoodie,2858.2519,-6731.2840


,date
artist,
2 Chainz,NaT
21 Savage,NaT
3 Doors Down,NaT
3LAU,NaT
5 Seconds of Summer,NaT
50 Cent,NaT
6LACK,NaT
6ix9ine,NaT
A Boogie Wit da Hoodie,NaT
